In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
cid = os.getenv("CLIENT_ID")
secret = os.getenv("CLIENT_SECRET")
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
for i in range (0,100):
    playlists_data=sp.search(q='rewin',type='playlist')


In [4]:
playlists_data

{'playlists': {'href': 'https://api.spotify.com/v1/search?query=rewin&type=playlist&offset=0&limit=10',
  'items': [{'collaborative': False,
    'description': 'Remember what you listened to in 2010? Rewind and rediscover your favorites.',
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DXc6IFF23C9jj'},
    'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DXc6IFF23C9jj',
    'id': '37i9dQZF1DXc6IFF23C9jj',
    'images': [{'height': None,
      'url': 'https://i.scdn.co/image/ab67706f0000000327ba1078080355421d1a49e2',
      'width': None}],
    'name': 'Rewind - The Sound of 2010',
    'owner': {'display_name': 'Spotify',
     'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
     'href': 'https://api.spotify.com/v1/users/spotify',
     'id': 'spotify',
     'type': 'user',
     'uri': 'spotify:user:spotify'},
    'primary_color': None,
    'public': None,
    'snapshot_id': 'MTU5NTYwNzA2MSwwMDAwMDAwMGQ0MWQ4Y2Q5OGYwMGIyMDRlOTgwMDk5

In [5]:
ids = [item["id"] for item in playlists_data["playlists"]["items"]]
ids

['37i9dQZF1DXc6IFF23C9jj',
 '37i9dQZF1DX0yEZaMOXna3',
 '37i9dQZF1DXe2bobNYDtW8',
 '37i9dQZF1DXcagnSNtrGuJ',
 '37i9dQZF1DX9ukdrXQLJGZ',
 '37i9dQZF1DWTE7dVUebpUW',
 '37i9dQZF1DX0h0QnLkMBl4',
 '37i9dQZF1DX3Sp0P28SIer',
 '37i9dQZF1DX0s5kDXi1oC5',
 '37i9dQZF1DXco4ODkIraMD']

In [6]:
sp.playlist_tracks('37i9dQZF1DXc6IFF23C9jj')


{'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DXc6IFF23C9jj/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2020-06-19T08:36:01Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
    'href': 'https://api.spotify.com/v1/users/',
    'id': '',
    'type': 'user',
    'uri': 'spotify:user:'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7dGJo4pcD2V6oG8kP0tJRR'},
       'href': 'https://api.spotify.com/v1/artists/7dGJo4pcD2V6oG8kP0tJRR',
       'id': '7dGJo4pcD2V6oG8kP0tJRR',
       'name': 'Eminem',
       'type': 'artist',
       'uri': 'spotify:artist:7dGJo4pcD2V6oG8kP0tJRR'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
      'CH',
      'CL',
    

In [7]:
def analyze_playlist(*args):
    
    
    playlist_features_list = ["artist","track_name","track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
   
    
    playlist_tracks = sp.playlist_tracks(*args)["items"]
    for track in playlist_tracks:
        
        playlist_features = {}
        
        playlist_features["artist"] = track["track"]["artists"][0]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
    
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[3:]:
            playlist_features[feature] = audio_features[feature]
        
        
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [21]:
playlist_df2010=analyze_playlist('37i9dQZF1DXc6IFF23C9jj')
playlist_df2010.head()

,artist,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms
0,Eminem,Love The Way You Lie,15JINEqzVMv3SvJTAXAKED,0.749,0.925,10,-5.034,1,0.2270,0,0.5200,0.641,86.989,263373
1,Kesha,TiK ToK,0HPD5WQqrq7wPWR7P7Dw1i,0.755,0.837,2,-2.718,0,0.1420,0,0.2890,0.714,120.028,199693
2,Lady Gaga,Bad Romance,0SiywuOBRcynK0uKGWdCnn,0.696,0.921,0,-3.755,1,0.0363,5.24e-05,0.0842,0.714,119.001,294573
3,Train,"Hey, Soul Sister",4HlFJV71xXKIGcU3kRyttv,0.673,0.886,1,-4.440,0,0.0431,0,0.0826,0.795,97.012,216773
4,Robyn,Dancing On My Own,5vd5Xnm203ZmzulaK49zJw,0.687,0.865,6,-4.663,1,0.0349,0.225,0.0966,0.261,117.015,288670


In [9]:
playlist_df2011=analyze_playlist('37i9dQZF1DX0yEZaMOXna3')


In [10]:
playlist_df2012=analyze_playlist('37i9dQZF1DXe2bobNYDtW8')


In [11]:
playlist_df2013=analyze_playlist('37i9dQZF1DXcagnSNtrGuJ')


In [12]:
playlist_df2014=analyze_playlist('37i9dQZF1DX9ukdrXQLJGZ')


In [13]:
playlist_df2015=analyze_playlist('37i9dQZF1DWTE7dVUebpUW')


In [14]:
playlist_df2016=analyze_playlist('37i9dQZF1DX0h0QnLkMBl4')


In [15]:
playlist_df2017=analyze_playlist('37i9dQZF1DX3Sp0P28SIer')


In [16]:
playlist_df2018=analyze_playlist('37i9dQZF1DX0s5kDXi1oC5')


In [17]:
playlist_df2019=analyze_playlist('37i9dQZF1DXco4ODkIraMD')


In [18]:
all_dataframes=[playlist_df2010,playlist_df2011,playlist_df2012,playlist_df2013,playlist_df2014,playlist_df2015,playlist_df2016,playlist_df2017,playlist_df2018,playlist_df2019]


In [19]:
playlist= pd.concat(all_dataframes)
playlist


,artist,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms
0,Eminem,Love The Way You Lie,15JINEqzVMv3SvJTAXAKED,0.749,0.925,10,-5.034,1,0.2270,0,0.5200,0.641,86.989,263373
1,Kesha,TiK ToK,0HPD5WQqrq7wPWR7P7Dw1i,0.755,0.837,2,-2.718,0,0.1420,0,0.2890,0.714,120.028,199693
2,Lady Gaga,Bad Romance,0SiywuOBRcynK0uKGWdCnn,0.696,0.921,0,-3.755,1,0.0363,5.24e-05,0.0842,0.714,119.001,294573
3,Train,"Hey, Soul Sister",4HlFJV71xXKIGcU3kRyttv,0.673,0.886,1,-4.440,0,0.0431,0,0.0826,0.795,97.012,216773
4,Robyn,Dancing On My Own,5vd5Xnm203ZmzulaK49zJw,0.687,0.865,6,-4.663,1,0.0349,0.225,0.0966,0.261,117.015,288670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Nacho,Báilame,7lRsNbdOGykkMAgsqs4R1C,0.817,0.869,6,-4.791,1,0.1800,0.00226,0.0497,0.561,96.029,207200
71,Daddy Yankee,Hula Hoop,7ztcWZ0EB6akSzlb8BUaqG,0.798,0.818,6,-4.628,1,0.0801,0.00315,0.0942,0.951,176.000,236493
72,KAROL G,A Ella,4N5qvaVICVIRxBXGW3FwGa,0.730,0.769,2,-4.936,1,0.1780,2.06e-05,0.1360,0.594,181.931,198720
73,Baby Rasta & Gringo,Amor Prohibido,28QkqSjl1Pi0niDc20RDl0,0.753,0.854,8,-6.177,1,0.0606,3.71e-06,0.0667,0.693,87.017,213000


In [20]:
playlist.to_csv('../OUTPUT/playlist.csv')

In [ ]:
#cprueba